<h1 align='center'> ⚖ Weighted Average Ensembling  🏋️‍♀️</h1>

This notebook is a model ensembling notebook as part of Tab Hack 2.0, a Machine Learning Hackathon organized by IITG.ai, The Artificial Intelligence Community of IIT Guwahati.


For more such hackathons and everything AI, do follow IITG.ai on these socials:
    
* [Linkedin](https://www.linkedin.com/company/iitg-ai/)
* [Website](https://www.iitg.ac.in/sa/ai/#/)
* [Instagram](https://www.instagram.com/iitg.ai/)

### 🎶 Optimization based Weighted Average Ensembling
* An alternative to searching for weight values is to use a directed optimization process.
* Optimization is a search process, but instead of sampling the space of possible solutions randomly or exhaustively, the search process uses any available information to make the next step in the search, such as toward a set of weights that has lower error.
* We use the **Scipy** Implementation of the **Differential Evolution** method to find the global minimum of a multivariate function.

## 📦 Importing Libraries

In [1]:
# Basic Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Time based imports
import datetime
from time import time

# The cool stuff
import tensorflow as tf
import scipy
from sklearn.metrics import f1_score, accuracy_score
from colorama import Fore, Back, Style

## 🔃 Data Loading

In [2]:
train = pd.read_csv('../input/tab-hack-20/Week8_train.csv')
test = pd.read_csv('../input/tab-hack-20/Week8_test.csv')

In [3]:
id_col = test['id']
feature_cols = [col for col in train.columns.tolist() if col not in ['TARGET']]
target_cols = ['TARGET']

## 👨‍🔬 The ensembling begins...

In [4]:
oof_paths = ['../input/oof-preds/oof_cat.npy',
             '../input/oof-preds/oof_cat (1).npy',
             '../input/oof-preds/oof_lgb.npy',
             '../input/oof-preds/oof_lgb (1).npy',
             '../input/oof-preds/oof_lgb (2).npy',
             '../input/oof-preds/oof_xgb.npy',
             '../input/oof-preds/oof_xgb (2).npy',
             '../input/oof-preds/oof_xgb (3).npy']

pred_paths = ['../input/oof-preds/pred_cat.npy',
             '../input/oof-preds/pred_cat (1).npy',
             '../input/oof-preds/pred_lgb.npy',
             '../input/oof-preds/pred_lgb (1).npy',
             '../input/oof-preds/pred_lgb (2).npy',
              '../input/oof-preds/pred_xgb.npy',
              '../input/oof-preds/pred_xgb (2).npy',
             '../input/oof-preds/pred_xgb (3).npy']

oof_comp = np.zeros((len(oof_paths), len(train[target_cols]), 2))
pred_comp = np.zeros((len(pred_paths), len(test), 2))

for i in range(len(oof_paths)):
    oof_comp[i, :, :] = np.load(oof_paths[i])
    
for i in range(len(pred_paths)):
    pred_comp[i, :, :] = np.load(pred_paths[i])
    
y_true = (train[target_cols].values).reshape(-1,1)
y_true_hot = np.array(tf.one_hot(y_true, depth=2)).reshape(-1,2)


def f1score(oof):
    predictions = oof.argmax(axis=1)
    return f1_score(train[target_cols], predictions, average='macro')

# return negative of f1 to minimize
def func_numpy_metric(weights):
    oof_blend = np.tensordot(weights, oof_comp, axes=(0,0))
    return -f1score(oof_blend)

2022-09-10 10:44:37.973039: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
%%time

f1_scores = {}

for i in range(len(oof_paths)):
    score_oof = f1score(oof_comp[i, ...])
    f1_scores[oof_paths[i]] = score_oof
    print(f'{oof_paths[i]} CV:\t\t\t',f'{Fore.YELLOW}{Style.BRIGHT}',score_oof,f'{Style.RESET_ALL}')
print('-' * 80)

../input/oof-preds/oof_cat.npy CV:			  0.6525343017800713 
../input/oof-preds/oof_cat (1).npy CV:			  0.6523905036267263 
../input/oof-preds/oof_lgb.npy CV:			  0.6525829407803301 
../input/oof-preds/oof_lgb (1).npy CV:			  0.6529750791361038 
../input/oof-preds/oof_lgb (2).npy CV:			  0.652263575396941 
../input/oof-preds/oof_xgb.npy CV:			  0.6531533715660178 
../input/oof-preds/oof_xgb (2).npy CV:			  0.6524598483803724 
../input/oof-preds/oof_xgb (3).npy CV:			  0.6527397009792669 
--------------------------------------------------------------------------------
CPU times: user 206 ms, sys: 0 ns, total: 206 ms
Wall time: 208 ms


In [6]:
tolerance = 1e-3
bounds = [(0, 1) for _ in range(oof_comp.shape[0])]
init_guess = [1 / oof_comp.shape[0]] * oof_comp.shape[0]
print(f'{Fore.RED}{Style.BRIGHT}Inital Blend OOF:', func_numpy_metric(init_guess),f'{Style.RESET_ALL}')
start_time = time()

# global optimization of ensemble weights
res_scipy = scipy.optimize.differential_evolution(func = func_numpy_metric,
                                                 bounds=bounds,
                                                 disp=True, 
                                                 tol=tolerance)

print(f'{Fore.GREEN}{Style.BRIGHT}Optimised Blend OOF:', res_scipy.fun)
print(f'{Style.RESET_ALL}Optimised Weights:', res_scipy.x)

# get the chosen weights
weights = res_scipy.x

Inital Blend OOF: -0.6538948214881087 
differential_evolution step 1: f(x)= -0.654247
Optimised Blend OOF: -0.6542467000583063
Optimised Weights: [0.13067212 0.57876246 0.34823518 0.23581806 0.02251458 0.18558999
 0.57732874 0.26721049]


In [7]:
# using optimized weights for ensembling 
def blend_preds(weights):
    for i in range(pred_comp.shape[0]):
        pred_blend = np.tensordot(weights, pred_comp, axes=(0,0))
        
    return pred_blend

pred_blend = blend_preds(weights)

## Submission

In [8]:
final_preds = pred_blend.argmax(axis=1)
pred_csv = pd.DataFrame(final_preds.reshape(-1), columns=['TARGET'] )
pred_csv['id'] = id_col
pred_csv

,TARGET,id
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
...,...,...
110894,0,110894
110895,0,110895
110896,1,110896
110897,0,110897


In [9]:
pred_csv.to_csv('submission.csv',index = False)